读取数据

In [1]:
import pandas as pd
import numpy as np
import csv

seasons = pd.read_csv("./data/bangumi.csv", delimiter=",", encoding="utf-8")
episodes = pd.read_csv("./data/episode.csv", delimiter=",", encoding="utf-8")
danmaku_complete = pd.read_csv("./data/danmaku_complete.csv", delimiter="\t", encoding="utf-8", quoting=csv.QUOTE_NONE, low_memory=False)
danmaku_complete = danmaku_complete.fillna(0)

选出候选的episode中的弹幕

In [2]:
danmaku_select = danmakus[danmakus['episode_id'] == 173248]

NameError: name 'danmakus' is not defined

导入embedding模型

In [1]:
from gensim.models.word2vec import Word2Vec
word_model = Word2Vec.load("./models/dm_word2vec_200.model")
word_dim = 200
print(len(word_model.wv.vocab))

166544


分词预处理方法

In [ ]:
import jieba.posseg as segtool
import re

ACCEPTABLE_TYPE = {'n', 't', 's', 'f', 'v', 'a', 'b', 'z', 'e', 'y', 'o'}
REPLACE_DICT = {
    "233+": "233",
    "666+": "666"
}

def check_type(word_type):
    if word_type[0] in ACCEPTABLE_TYPE:
        return True
    else:
        return False

def check_replace(word):
    for item in REPLACE_DICT.keys():
        pattern = re.compile(item)
        if re.match(pattern, word) is not None:
            new_word = REPLACE_DICT[item]
            return new_word
    return word

def word_segment(content):
    words = []
    results = segtool.cut(content)
    for result in results:
        result.word = check_replace(result.word)
        if check_type(result.flag):
            words.append(result.word)
    return words

窗口划分

In [ ]:
def window_slice(danmaku, start, end):
    window_danmaku = danmaku[(start <= danmaku.playback_time) & (danmaku.playback_time<= end)]
    return window_danmaku.sort_values(by='playback_time')

selected_window = window_slice(danmaku_select, 400, 410)

获取特征

In [ ]:
danmaku_feature = {}
for index, row in selected_window.iterrows():
    words = word_split(row['content'])
    count = 0
    sum = np.zeros(word_dim)
    if len(words) == 0:
        continue
    for word in words:
        if word in word_model.wv.vocab:
            sum += word_model.wv[word]
            count += 1
    if count > 0:
        danmaku_feature[row['tsc_raw_id']] = sum/count
len(danmaku_feature)

比较相似度，画出矩阵图

In [ ]:
from scipy.spatial.distance import pdist

num = len(danmaku_feature)
sim_matrix = np.zeros((num, num), dtype=float)
x_index = 0
for index, row in selected_window.iterrows():
    dmid = row['tsc_raw_id']
    if dmid in danmaku_feature:
        x_feature = danmaku_feature[dmid]
        y_index = 0
        for index_, row_ in selected_window.iterrows():
            dmid_ = row_['tsc_raw_id']
            if dmid_ in danmaku_feature:
                y_feature = danmaku_feature[dmid_]
                sim_matrix[x_index, y_index] = pdist(np.vstack([x_feature,y_feature]),'cosine')
                y_index += 1
        x_index += 1       

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (15.0, 8.0)

plt.matshow(sim_matrix, cmap='hot')
plt.colorbar()
plt.show()

随机投影

In [ ]:
num = len(danmaku_feature)
rand = np.random.RandomState(42) 
X = np.zeros((num, word_dim), dtype=float)
y = np.zeros(num, dtype=int)
x_index = 0
for index, row in selected_window.iterrows():
    dmid = row['tsc_raw_id']
    if dmid in danmaku_feature:
        x_feature = danmaku_feature[dmid]
        X[x_index] = x_feature
        flag = danmaku_complete[danmaku_complete['tsc_raw_id'] == dmid]['block_level'].iloc[0]
        y[x_index] = int(flag)
        x_index += 1

In [ ]:
i, j = rand.randint(X.shape[1], size=2)
plt.scatter(X[:, i], X[:, j], c=y)
plt.show()

In [ ]:
from lpproj import LocalityPreservingProjection
lpp = LocalityPreservingProjection(n_components=2)

X_2D = lpp.fit_transform(X)
plt.scatter(X_2D[:, 0], X_2D[:, 1])
plt.show()